In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
transform=transforms.ToTensor()

In [0]:
trainset=datasets.FashionMNIST(root='./FashionMNISTData',train=True,transform=transform,download=True)

0it [00:00, ?it/s]

26427392it [00:01, 14717465.25it/s]                             


Extracting ./FashionMNISTData/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNISTData/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 102064.14it/s]           
0it [00:00, ?it/s]

Extracting ./FashionMNISTData/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNISTData/FashionMNIST/raw


4423680it [00:01, 4283655.27it/s]                             
0it [00:00, ?it/s]

Extracting ./FashionMNISTData/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNISTData/FashionMNIST/raw


8192it [00:00, 37365.37it/s]            

Extracting ./FashionMNISTData/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNISTData/FashionMNIST/raw
Processing...
Done!


In [0]:
trainloader=torch.utils.data.DataLoader(trainset,batch_size=1,shuffle=True)

In [0]:
testset=datasets.FashionMNIST(root='./FashionMNISTData',train=False,transform=transform,download=True)

In [0]:
testloader=torch.utils.data.DataLoader(testset,batch_size=1,shuffle=False)

In [0]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [0]:
images.shape

torch.Size([1, 1, 28, 28])

In [0]:
X_train,Y_train=torch.load('FashionMNISTData/FashionMNIST/processed/training.pt')
X_test,Y_test=torch.load('FashionMNISTData/FashionMNIST/processed/test.pt')

In [0]:
X_train=X_train.reshape(-1,1,28,28)
X_test=X_test.reshape(-1,1,28,28)

In [0]:
X_test.shape

torch.Size([10000, 1, 28, 28])

In [0]:
device=torch.device("cuda")

In [0]:
print(device)

cuda


In [0]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3),
            nn.MaxPool2d(2, 2)
        )
        self.fc=nn.Sequential(
            nn.Linear(64 * 3 * 3, 96),
            nn.ReLU(),
            nn.Linear(96, 16),
            nn.ReLU(),
            nn.Linear(16, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 64 * 3 * 3)
        x = self.fc(x)
        return x


net = ConvNet()

In [0]:
def fit(X,Y,model,opt,loss_fn,epochs):
  for epoch in range(epochs):
    loss=loss_fn(model(X),Y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    if(epoch%10==0):
      print(epoch,'/',5000,' :  ',loss.item())
  return loss.item()

In [0]:
model=net
opt=optim.Adam(net.parameters(),lr=0.001,betas=(0.8,0.888))
loss_fn=nn.CrossEntropyLoss()

In [0]:
X_train=X_train.to(device,dtype=torch.float)
Y_train=Y_train.to(device)
X_test=X_test.to(device,dtype=torch.float)
Y_test=Y_test.to(device)
model=model.to(device)

In [0]:
%%time
print(fit(X_train,Y_train,model,opt,loss_fn,epochs=5000))

0 / 5000  :   2.3160932064056396
10 / 5000  :   2.0752251148223877
20 / 5000  :   1.8954331874847412
30 / 5000  :   1.855053424835205
40 / 5000  :   1.836181402206421
50 / 5000  :   1.8316477537155151
60 / 5000  :   1.8205456733703613
70 / 5000  :   1.8092879056930542
80 / 5000  :   1.8077423572540283
90 / 5000  :   1.8015042543411255
100 / 5000  :   1.7988921403884888
110 / 5000  :   1.800792932510376
120 / 5000  :   1.792201042175293
130 / 5000  :   1.722718358039856
140 / 5000  :   1.6976895332336426
150 / 5000  :   1.6886628866195679
160 / 5000  :   1.6756664514541626
170 / 5000  :   1.662192940711975
180 / 5000  :   1.6603429317474365
190 / 5000  :   1.667536735534668
200 / 5000  :   1.6511003971099854
210 / 5000  :   1.6531826257705688
220 / 5000  :   1.6434276103973389
230 / 5000  :   1.6442184448242188
240 / 5000  :   1.6469258069992065
250 / 5000  :   1.6399940252304077
260 / 5000  :   1.6374845504760742
270 / 5000  :   1.6370487213134766
280 / 5000  :   1.6292728185653687
290

KeyboardInterrupt: ignored

In [0]:
hat=[]
T=model(X_test)
for i in range(T.shape[0]):
  nmax=0
  for j in range(10):
    if T[i][j].item()>nmax:
      nmax=T[i][j].item()
      imax=j
  hat.append(imax)
Y_hat=torch.tensor(hat)

In [0]:
Y_hat.shape

torch.Size([10000])

In [0]:
print('Test set accuracy: ',accuracy_score(Y_test.detach().cpu(),Y_hat.cpu()))

Test set accuracy:  0.8159


In [0]:
torch.save(model.state_dict(), './statesBibhuti.pth')

In [0]:
hat=[]
T=model(X_train)
for i in range(T.shape[0]):
  nmax=0
  for j in range(10):
    if T[i][j].item()>nmax:
      nmax=T[i][j].item()
      imax=j
  hat.append(imax)
Y_hat=torch.tensor(hat)

In [0]:
print('Train set accuracy: ',accuracy_score(Y_train.detach().cpu(),Y_hat.cpu()))

Train set accuracy:  0.8763166666666666
